In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree,ensemble,metrics

from rule import Rule
from rule_extraction import rule_extract

## 数据准备

In [2]:
# fetch dataset
data = pd.read_csv('./dataset/titanic.csv', usecols = ['Age', 'Fare', 'Survived'])

# drop NA records since most Tree algorithm cannot handle
data.dropna(inplace=True)

# split training/test sets
X_train, X_test, y_train, y_test = train_test_split(data[['Age', 'Fare']], data.Survived, test_size=0.2, random_state=0)

# dataset shape
print(X_train.shape, X_test.shape)


(571, 2) (143, 2)


## 单颗决策树

In [3]:
# 单颗决策树模型
# 模型API参考 http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

model_tree_clf = tree.DecisionTreeClassifier(criterion='gini',max_depth=3)
model_tree_clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [4]:
# model performance on test set
y_pred_test = model_tree_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[67 12]
 [33 31]]


In [5]:
rule_extract(model_tree_clf,X_test,y_test)

[('Age > 8.5 and Fare <= 51.931251525878906 and Fare > 10.335399627685547',
  ('recall for class 1 is: 0.40625',
   'prec for class 1 is: 0.4727272727272727',
   'recall for class 0 is: 0.6329113924050633',
   'prec for class 0 is: 0.42735042735042733')),
 ('Age > 29.5 and Fare > 51.931251525878906',
  ('recall for class 1 is: 0.25',
   'prec for class 1 is: 0.6956521739130435',
   'recall for class 0 is: 0.9113924050632911',
   'prec for class 0 is: 0.5669291338582677')),
 ('Age <= 29.5 and Age > 17.5 and Fare > 51.931251525878906',
  ('recall for class 1 is: 0.109375',
   'prec for class 1 is: 0.6363636363636364',
   'recall for class 0 is: 0.9493670886075949',
   'prec for class 0 is: 0.5514705882352942')),
 ('Age <= 8.5 and Fare <= 27.825000762939453',
  ('recall for class 1 is: 0.09375',
   'prec for class 1 is: 1.0',
   'recall for class 0 is: 1.0',
   'prec for class 0 is: 0.5766423357664233')),
 ('Age > 8.5 and Fare <= 10.335399627685547',
  ('recall for class 1 is: 0.078125',


## 随机森林

In [6]:
model_RF_clf = ensemble.RandomForestClassifier(max_depth=3,n_estimators=2)
model_RF_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [7]:
# model performance on test set
y_pred_test = model_RF_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[68 11]
 [40 24]]


In [8]:
rule_extract(model_RF_clf,X_test,y_test)

[('Fare <= 56.197898864746094 and Fare > 9.416650772094727',
  ('recall for class 1 is: 0.5625',
   'prec for class 1 is: 0.4864864864864865',
   'recall for class 0 is: 0.5189873417721519',
   'prec for class 0 is: 0.38317757009345793')),
 ('Fare > 56.197898864746094',
  ('recall for class 1 is: 0.359375',
   'prec for class 1 is: 0.696969696969697',
   'recall for class 0 is: 0.8734177215189873',
   'prec for class 0 is: 0.575')),
 ('Fare <= 15.625 and Fare > 7.227099895477295',
  ('recall for class 1 is: 0.296875',
   'prec for class 1 is: 0.30158730158730157',
   'recall for class 0 is: 0.44303797468354433',
   'prec for class 0 is: 0.28225806451612906')),
 ('Fare > 75.1146011352539',
  ('recall for class 1 is: 0.28125',
   'prec for class 1 is: 0.72',
   'recall for class 0 is: 0.9113924050632911',
   'prec for class 0 is: 0.576')),
 ('Fare <= 48.20000076293945 and Fare > 16.0',
  ('recall for class 1 is: 0.28125',
   'prec for class 1 is: 0.5294117647058824',
   'recall for class

## BaggingClassifier

In [9]:
model_bagging_clf = ensemble.BaggingClassifier(
                base_estimator=tree.DecisionTreeClassifier(max_depth=3),
                n_estimators=2,
                n_jobs=-1,
                random_state=0)
model_bagging_clf.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=2, n_jobs=-1, oob_score=False,
         random_state=0, verbose=0, warm_start=False)

In [10]:
# model performance on test set
y_pred_test = model_bagging_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[73  6]
 [41 23]]


In [11]:
rule_extract(model_bagging_clf,X_test,y_test)

[('Fare <= 44.239601135253906 and Fare > 10.824999809265137',
  ('recall for class 1 is: 0.46875',
   'prec for class 1 is: 0.5',
   'recall for class 0 is: 0.620253164556962',
   'prec for class 0 is: 0.4336283185840708')),
 ('Fare <= 82.66455078125 and Fare > 19.856250762939453',
  ('recall for class 1 is: 0.40625',
   'prec for class 1 is: 0.5306122448979592',
   'recall for class 0 is: 0.7088607594936709',
   'prec for class 0 is: 0.47863247863247865')),
 ('Age <= 44.5 and Age > 1.5 and Fare <= 18.375',
  ('recall for class 1 is: 0.265625',
   'prec for class 1 is: 0.265625',
   'recall for class 0 is: 0.40506329113924056',
   'prec for class 0 is: 0.25396825396825395')),
 ('Age <= 45.0 and Age > 25.5 and Fare > 51.931251525878906',
  ('recall for class 1 is: 0.203125',
   'prec for class 1 is: 0.8125',
   'recall for class 0 is: 0.9620253164556962',
   'prec for class 0 is: 0.5846153846153846')),
 ('Age > 16.5 and Fare <= 10.824999809265137',
  ('recall for class 1 is: 0.125',
   

## 极端随机树

In [12]:
model_extratree_clf = ensemble.ExtraTreesClassifier(max_depth=3,n_estimators=2)
model_extratree_clf.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=3, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=2, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [13]:
# model performance on test set
y_pred_test = model_extratree_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[74  5]
 [56  8]]


In [14]:
rule_extract(model_extratree_clf,X_test,y_test)

[('Age <= 77.38182124642407 and Fare <= 105.26056261931572',
  ('recall for class 1 is: 0.875',
   'prec for class 1 is: 0.4307692307692308',
   'recall for class 0 is: 0.06329113924050633',
   'prec for class 0 is: 0.05747126436781609')),
 ('Age > 24.058929195029002 and Fare <= 185.55054721182938 and Fare > 17.874592264134076',
  ('recall for class 1 is: 0.421875',
   'prec for class 1 is: 0.6136363636363636',
   'recall for class 0 is: 0.7848101265822784',
   'prec for class 0 is: 0.5344827586206896')),
 ('Age <= 24.058929195029002 and Fare <= 185.55054721182938 and Fare > 17.874592264134076',
  ('recall for class 1 is: 0.21875',
   'prec for class 1 is: 0.6086956521739131',
   'recall for class 0 is: 0.8860759493670887',
   'prec for class 0 is: 0.5426356589147286')),
 ('Age > 23.579183918583364 and Fare <= 17.874592264134076 and Fare > 3.804449314728431',
  ('recall for class 1 is: 0.203125',
   'prec for class 1 is: 0.2765957446808511',
   'recall for class 0 is: 0.569620253164556